# Numerical approach for the reaction–diffusion problem to model the density of the tumor growth dynamics

## The aim of this worksheet is to solve the differential equation below
\begin{equation}
\dfrac{\partial c(x,t)}{\partial t} = \dfrac{\partial }{\partial x}\left(D(x)\dfrac{\partial c(x,t)}{\partial x} \right) + \rho c(x,t)\left(1- \dfrac{c(t,x)}{c_{max}}\right)
\end{equation}
and where the diffusion coefficient $D$
\begin{equation}
D(x) = \begin{split}
D_g = 0.13 mm^2/day \qquad\qquad  0\le x \le 7.5 \qquad \text{(gray region)} \\
D_w = 0.65 mm^2/day \qquad  7.5\le x \le 42.5 \qquad \text{(white region)}\\
D_g = 0.13 mm^2/day \qquad\qquad  42.5\le x \le 50 \qquad \text{(gray region)}
\end{split}
\end{equation}
with the Neumann boundary conditions (zero-flux)
\begin{equation}
c_x(0,t)=0 \qquad \qquad \qquad  c_x(50,t)=0
\end{equation}
and the initial condition is taken as Gaussian initial tumor profile
\begin{equation}
c(x,0)= g(x) = \dfrac{1}{\sqrt{2\pi}\varepsilon}e^{-\frac{1}{2}\left(\frac{x-x_0}{\varepsilon}\right)^2}
\end{equation}
where $c = c ( t , x )$ denoted the tumor concentration of glioma cells at time $t$ and spatial location $x$ and $x_0 = 25 mm$ (the middle of the considered interval) and $ε = 0 . 01$. Here, $\rho$ is the net proliferation rate in units $day^{-1}$ and Fickian diffusion has been used to quantify the random motility of a variety of invading cells (Cozens-Roberts et al). A factor of $5$, $D_w = 5 D_g$, was used by Swanson et al. but this may vary from patient to patient.

In [130]:
import numpy as np 
import matplotlib.pyplot as plt

In [142]:
# intialisation 
a= 0 ; b = 50  # space dimensions 
ti=0 ; tf= 50  # time observation
dt = 0.01
dx = 0.5
M = int(b/dx) # length of the space grid [x_0, x_1, ... , x_M]
P =  int(tf/dt) # length of the time grid [t_0, t1, ... , t_P]
x = np.linspace(a,b,M+1)
t = np.linspace(ti, tf, P+1)
c = np.zeros((M+1,P+1))
D_ = np.zeros(M) # table which contains the all the values of D(x), D_[i]=D_{i+1/2}=D((x[i]+x[i+1])/2)
                 # it is easy to understand that D_[i-1]=D_{i-1/2}...

# constants initialisation
eps = 0.01
x0 = 25
Dg = 0.13
Dw = 5*Dg
rho = 0.012
cmax = 62.5 
#cmax = 39.89

In [143]:
# Initial condition
def g(x_):
    result = np.exp(-0.5*((x_-x0)/eps)**2)/(eps*np.sqrt(2*np.pi)) 
    return result

c[:,0]=g(x)

# diffusion coefficient D
def D(x_):
    if x_>=0 and x_<=7.5:
        return Dg
    elif x_>=7.5 and x_<=42.5:
        return Dw
    elif x_>=42.5 and x_<=50:
        return Dg
    else :
        print("Out of the zone, the arg of D not belong to [0,50]")

for i in range(len(x)-1):
    D_[i] = D((x[i]+x[i+1])/2)

In [144]:
alpha = dt/(2*dx**2)
beta = dt*rho/2
gamma = dt*rho/(2*cmax)
def F(c_, b):
    result=[]
    for i in range(1,len(c_)-1):
        result=[*result, alpha*D_[i-1]*c_[i-1]+ alpha*D_[i]*c_[i+1] - (alpha*(D_[i-1]+D_[i]) -
                                                        beta + 1)*c_[i] - gamma*c_[i]**2 + b[i] ]
    return np.array(result)

def delta(m:int,p:int)->int:
    return 1 if m==p else 0

def dF_matrix(c_):
    result=np.zeros((len(c_)-2,len(c_)-2))
    for i in range(1,len(c_)-1):
        for j in range(1,len(c_)-1):
            result[i-1,j-1] = alpha*D_[i-1]*delta(i-1,j)+ alpha*D_[i]*delta(i+1,j) - (alpha*(D_[i-1]+D_[i]) -
                                                        beta + 1)*delta(i,j) - 2*gamma*c_[i]*delta(i,j)
    return result
    

In [145]:
# Use Newton's method to solve systems of nonlinear algebraic equations.
def Newton_system(F, dF_matrix, c_, b, err):
    # Solve nonlinear system F=0 by Newton's method.
    # dF_matrix is the Jacobian of F. Both F and dF_matrix must be functions of c.
    # At input, c holds the start value. The iteration continues
    # until ||F|| < err.

    F_value = F(c_, b)
    dF_matrix_value = dF_matrix(c_)
    F_norm = np.linalg.norm(F_value, ord=2)  #l2 norm of vector
    iteration_counter = 0
    while abs(F_norm) > err and iteration_counter < 100:
        delta_c = np.linalg.solve(dF_matrix_value, -F_value)
        c_[1:len(F_value)+1] = c_[1:len(F_value)+1] + delta_c
        c_[0] = c_[2]    # boundary conditions
        c_[M] = c_[M-2]  # boundary conditions
        F_value = F(c_, b)
        dF_matrix_value = dF_matrix(c_)
        F_norm = np.linalg.norm(F_value, ord=2)
        iteration_counter = iteration_counter + 1

    # Here, either a solution is found, or too many iterations
    if abs(F_norm) > err:
        iteration_counter = -1
        print('solutions no found: no convergence')
    return c_

In [146]:
err = 0.001
for n in range(1,P+1):
    # print('for the time t='+str(n))
    # first guest for the Newton method
    c[:,n]=c[:,0]
    #use of the Newton method
    c[:,n] = Newton_system(F, dF_matrix, c[:,n], c[:,n-1], err)

In [151]:
for (t_obs, marker) in  [(1,'*'), (5, 'o'), (10, '--'),(50, '-.')]:
    nt = int(t_obs/dt) 
    plt.plot(x,c[:,nt], marker, label='for $t=$'+str(t_obs)+' days' )
plt.title(r'The values of the tumor concentration $c(t,x)$ of glioma cells versus $x$ ')
plt.grid(True)
plt.xlabel('x')
plt.ylabel('$c(x,t)$')
#plt.legend(bbox_to_anchor=(1,1))
plt.legend(loc='best')
plt.savefig('figure_new.png')
plt.show()